# MERRA2 SLV+Radiation Hourly Granularity Test
- Download
- Trim
- Create Cutout
- Generate Solar Outputs

## 1. Import and Download Data

In [1]:
import calendar
import geodata
import xarray as xr
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)
DS_hourly = geodata.Dataset(module="merra2",
					 years=slice(1997, 1997),
					 months=slice(1,1),
                     weather_data_config = "slv_radiation_hourly")  

INFO:geodata.dataset:Directory /Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2 found, checking for completeness.
INFO:geodata.dataset:Directory complete.


In [2]:
if DS_hourly.prepared == False:
	DS_hourly.get_data()

## 2. Trim Variables

In [ ]:
## Variables before trimming
ds = xr.open_dataset(DS_hourly.downloadedFiles[0][1])
ds.data_vars

In [ ]:
DS_hourly.trim_variables()

In [4]:
## Variables after trimming
ds = xr.open_dataset(DS_hourly.downloadedFiles[0][1])
ds.data_vars

Data variables:
    albedo   (time, lat, lon) float32 ...
    swgdn    (time, lat, lon) float32 ...
    swtdn    (time, lat, lon) float32 ...
    t2m      (time, lat, lon) float32 ...

## 3. Create Cutout

In [5]:
hourly_cutout = geodata.Cutout(name="hourly-test-cutout",
                       module="merra2",
                       weather_data_config="slv_radiation_hourly",
                       xs=slice(138.5, 139.5),
                       ys=slice(35, 36),
                       years=slice(1997, 1997),
                       months=slice(1,1))

INFO:geodata.cutout:Cutout (hourly-test-cutout, /Users/williamhonaker/Desktop/davidson/data_for_geodata/cutouts) not found or incomplete.


In [6]:
hourly_cutout.prepare()

INFO:geodata.preparation:Starting preparation of cutout 'hourly-test-cutout'
INFO:geodata.datasets.merra2:MultiIndex([(1997, 1)],
           names=['year', 'month'])
INFO:geodata.datasets.merra2:[(1997, 1, 1), (1997, 1, 2), (1997, 1, 3), (1997, 1, 4), (1997, 1, 5), (1997, 1, 6), (1997, 1, 7), (1997, 1, 8), (1997, 1, 9), (1997, 1, 10), (1997, 1, 11), (1997, 1, 12), (1997, 1, 13), (1997, 1, 14), (1997, 1, 15), (1997, 1, 16), (1997, 1, 17), (1997, 1, 18), (1997, 1, 19), (1997, 1, 20), (1997, 1, 21), (1997, 1, 22), (1997, 1, 23), (1997, 1, 24), (1997, 1, 25), (1997, 1, 26), (1997, 1, 27), (1997, 1, 28), (1997, 1, 29), (1997, 1, 30), (1997, 1, 31)]
INFO:geodata.preparation:31 tasks have been collected. Starting running them on all processors.
INFO:geodata.datasets.merra2:Opening `/Users/williamhonaker/Desktop/davidson/data_for_geodata/merra2/1997/01/MERRA2_200.tavg1_2d_slv_rad_Nx.19970101.nc4`
INFO:geodata.datasets.merra2:Opening `/Users/williamhonaker/Desktop/davidson/data_for_geodata/merr

In [8]:
# Verify Cutout Contents
hourly_cutout.__dict__

{'name': 'hourly-test-cutout',
 'cutout_dir': '/Users/williamhonaker/Desktop/davidson/data_for_geodata/cutouts/hourly-test-cutout',
 'prepared': True,
 'meta_append': 0,
 'config': 'slv_radiation_hourly',
 'dataset_module': <module 'geodata.datasets.merra2' from '/usr/local/lib/python3.7/site-packages/geodata-0.0.2-py3.7.egg/geodata/datasets/merra2.py'>,
 'meta': <xarray.Dataset>
 Dimensions:     (time: 744, x: 2, y: 3, year-month: 1)
 Coordinates:
   * time        (time) datetime64[ns] 1997-01-01T00:30:00 ... 1997-01-31T23:30:00
   * x           (x) float64 138.8 139.4
   * y           (y) float64 35.0 35.5 36.0
     lon         (x) float64 138.8 139.4
     lat         (y) float64 35.0 35.5 36.0
   * year-month  (year-month) MultiIndex
   - year        (year-month) int64 1997
   - month       (year-month) int64 1
 Data variables:
     *empty*
 Attributes:
     module:   merra2}

## 4. Generate Solar Output

In [10]:
ds_pv = geodata.convert.pv(
    hourly_cutout, 
    panel="KANENA", 
    orientation = "latitude_optimal"
    )
ds_pv

<xarray.DataArray 'AC power' (y: 3, time: 744, x: 2)>
array([[[0.28905639, 0.28089478],
        [0.43574266, 0.42219579],
        [0.53800452, 0.54373469],
        ...,
        [0.        , 0.        ],
        [0.0434896 , 0.04548016],
        [0.17568191, 0.18254081]],

       [[0.30820237, 0.29355579],
        [0.44990651, 0.41478911],
        [0.52704902, 0.49089815],
        ...,
        [0.        , 0.        ],
        [0.04282355, 0.04701791],
        [0.17506339, 0.17932855]],

       [[0.29296198, 0.26647445],
        [0.4266345 , 0.39563724],
        [0.49771324, 0.47503696],
        ...,
        [0.        , 0.        ],
        [0.04643929, 0.04982751],
        [0.17094576, 0.17261607]]])
Coordinates:
    lon      (x) float64 138.8 139.4
    lat      (y) float64 35.0 35.5 36.0
  * x        (x) float64 138.8 139.4
  * y        (y) float64 35.0 35.5 36.0
  * time     (time) datetime64[ns] 1997-01-01T00:30:00 ... 1997-01-31T23:30:00

In [11]:
df_pv = ds_pv.to_dataframe(name='pv')
df_pv

lon   lat        pv
y    time                x                               
35.0 1997-01-01 00:30:00 138.750  138.750  35.0  0.289056
                         139.375  139.375  35.0  0.280895
     1997-01-01 01:30:00 138.750  138.750  35.0  0.435743
                         139.375  139.375  35.0  0.422196
     1997-01-01 02:30:00 138.750  138.750  35.0  0.538005
...                                   ...   ...       ...
36.0 1997-01-31 21:30:00 139.375  139.375  36.0  0.000000
     1997-01-31 22:30:00 138.750  138.750  36.0  0.046439
                         139.375  139.375  36.0  0.049828
     1997-01-31 23:30:00 138.750  138.750  36.0  0.170946
                         139.375  139.375  36.0  0.172616

[4464 rows x 3 columns]

In [12]:
df_pv.to_csv('pv_test.csv')